In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/data")
df_rf_15 = pd.read_csv('df_rf_15.csv')
df_rf_20 = pd.read_csv('df_rf_20.csv')
df_rf_25 = pd.read_csv('df_rf_25.csv')
df_rf_30 = pd.read_csv('df_rf_30.csv')
df_rf_35 = pd.read_csv('df_rf_35.csv')
df_rf_40 = pd.read_csv('df_rf_40.csv')
df_rf_45 = pd.read_csv('df_rf_45.csv')
heroes= pd.read_csv('heroes.csv')
heroes = heroes.rename(columns = {'id':'hero_id'})
df2_allhero = pd.read_csv('df2_allhero.csv') # We use this to instantiate the embedding layer since thats how the embedding layers were created and i want to avoid any shenanigans like the hero_ids and their indexes getting jumbled due to ordering when they get mapped by hero_id_to_idx

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/")
%run LSTM_Model_Classes.ipynb
RMSE_model = torch.load('10step_5horizon_best_model_RMSE.pth')
MAE_model = torch.load('10step_5horizon_best_model_MAE.pth')
OK_model = torch.load('10step_5horizon_best_model_OK.pth')

In [ ]:
RMSE_embed_weights = RMSE_model['process_embedding.process_embeddings.weight']
MAE_embed_weights = MAE_model['process_embedding.process_embeddings.weight']
OK_embed_weights = OK_model['process_embedding.process_embeddings.weight']

In [ ]:
lookback = 10

RMSE_embedding = ProcessEmbedding(df2_allhero, embedding_dim=84, lookback=lookback)
RMSE_embedding.process_embeddings.weight = nn.Parameter(RMSE_embed_weights) # Load the old weights into the new embedding layer

MAE_embedding = ProcessEmbedding(df2_allhero, embedding_dim=84, lookback=lookback)
MAE_embedding.process_embeddings.weight = nn.Parameter(MAE_embed_weights)

OK_embedding = ProcessEmbedding(df2_allhero, embedding_dim=84, lookback=lookback)
OK_embedding.process_embeddings.weight = nn.Parameter(OK_embed_weights)

In [ ]:
columns_to_drop = df_rf_15.filter(regex=r'\d_gold$').columns #dropping columns that contain the time-series tensors
df_rf_15.drop(columns=columns_to_drop)

,match_id,rad_pos1,rad_pos2,rad_pos3,rad_pos4,rad_pos5,rad_pos1_last_gold,rad_pos2_last_gold,rad_pos3_last_gold,rad_pos4_last_gold,...,dire_pos4_last_gold,dire_pos5_last_gold,pos1_diff,pos2_diff,pos3_diff,pos4_diff,pos5_diff,team_diff,minute,winner
0,7517376613,25,8,84,26,5,6399,5383,5885,2609,...,2816,2692,445,-1062,-315,-207,-455,-1594,15,dire
1,7517420109,80,70,23,63,100,5917,6235,6524,2798,...,2866,2798,-3138,-41,1095,-68,-170,-2322,15,radiant
2,7517437309,10,104,19,45,64,6944,5849,5584,3521,...,2700,3026,254,279,941,821,-766,1529,15,dire
3,7517480603,1,33,11,100,30,7209,6843,6734,2371,...,3095,2296,1572,1184,1554,-724,475,4061,15,radiant
4,7517522012,69,10,76,83,86,8298,7366,6309,4413,...,3808,2856,1493,863,273,605,527,3761,15,dire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,7633793219,48,35,129,85,86,5762,7048,5086,4459,...,3276,2775,-1161,1099,-515,1183,44,650,15,dire
991,7633814106,18,137,98,5,27,7221,6252,6159,3541,...,3196,3610,2,-2171,126,345,-995,-2693,15,radiant
992,7633956117,70,96,23,31,123,5373,4127,4223,3500,...,3735,3792,-1872,-3323,-2267,-235,-1764,-9461,15,dire
993,7634024715,94,17,29,27,66,6347,6327,5070,2669,...,3175,3841,-1051,-612,-903,-506,-859,-3931,15,radiant


In [ ]:
df_rf_15.columns

Index(['match_id', 'rad_pos1', 'rad_pos2', 'rad_pos3', 'rad_pos4', 'rad_pos5',
       'rad_pos1_gold', 'rad_pos2_gold', 'rad_pos3_gold', 'rad_pos4_gold',
       'rad_pos5_gold', 'rad_pos1_last_gold', 'rad_pos2_last_gold',
       'rad_pos3_last_gold', 'rad_pos4_last_gold', 'rad_pos5_last_gold',
       'dire_pos1', 'dire_pos2', 'dire_pos3', 'dire_pos4', 'dire_pos5',
       'dire_pos1_gold', 'dire_pos2_gold', 'dire_pos3_gold', 'dire_pos4_gold',
       'dire_pos5_gold', 'dire_pos1_last_gold', 'dire_pos2_last_gold',
       'dire_pos3_last_gold', 'dire_pos4_last_gold', 'dire_pos5_last_gold',
       'pos1_diff', 'pos2_diff', 'pos3_diff', 'pos4_diff', 'pos5_diff',
       'team_diff', 'minute', 'winner'],
      dtype='object')

In [ ]:
def iterate_ids(id_array):
  # Takes in a numpy array, and feeds the values one by one into the embedding layer
  # Returns the embedding representation that can replace the column
  hero_embeddings_list = []
  for hero_id in id_array:
      embedding = RMSE_embedding(hero_id).detach().numpy()
      hero_embeddings_list.append(embedding)

  hero_embeddings_list = torch.stack(hero_embeddings_list)
  return hero_embeddings_list

In [ ]:
# We need to loop over the hero_id's since the ProcessEmbedding layer is expecting one hero_id at a time

# Below is commented out to show that depending on which dataframe we feed the hero_ids from we get different tensor embeddings.
# We will stick with the original embedding although it should technically work the same

# hero_embeddings = []
# for hero_id in heroes['hero_id']:
#     embedding = RMSE_embedding(hero_id)
#     hero_embeddings.append(embedding)

# hero_embeddings = torch.stack(hero_embeddings)

# hero_embeddings_df2 = []
# for hero_id in df2_allhero['hero_id']:
#     embedding = RMSE_embedding(hero_id)
#     hero_embeddings_df2.append(embedding)

# hero_embeddings_df2 = torch.stack(hero_embeddings_df2)

# hero_embeddings_df2 == hero_embeddings

In [ ]:
# embed_test = iterate_ids(df_rf_15['rad_pos1'].values)
# embed_test

In [ ]:
def get_hero_embedding_and_concat_features(df):
  # Extract the hero_ids for each position from your dataframe

  hero_columns = ['rad_pos1', 'rad_pos2', 'rad_pos3', 'rad_pos4', 'rad_pos5',
                'dire_pos1', 'dire_pos2', 'dire_pos3', 'dire_pos4', 'dire_pos5']

  hero_embeddings = {col: iterate_ids(df[col].values) for col in hero_columns}


  # Get the embedding vectors for each position using the trained embedding layer
  # We need to call another .detach().numpy() here since we were still getting the autograd error from PyTorch
  # We also needed to reshape the embeddings to remove an extra dimension, without the reshape(-1,84), their shapes were (995, 1, 84) now they're (995,84)

  rad_pos1_embeddings = hero_embeddings['rad_pos1'].detach().numpy().reshape(-1)
  #print(rad_pos1_embeddings.shape)
  rad_pos2_embeddings = hero_embeddings['rad_pos2'].detach().numpy().reshape(-1)
  #print(rad_pos2_embeddings.shape)
  rad_pos3_embeddings = hero_embeddings['rad_pos3'].detach().numpy().reshape(-1)
  #print(rad_pos3_embeddings.shape)
  rad_pos4_embeddings = hero_embeddings['rad_pos4'].detach().numpy().reshape(-1)
  #print(rad_pos4_embeddings.shape)
  rad_pos5_embeddings = hero_embeddings['rad_pos5'].detach().numpy().reshape(-1)
  #print(rad_pos5_embeddings.shape)

  dire_pos1_embeddings = hero_embeddings['dire_pos1'].detach().numpy().reshape(-1)
  #print(dire_pos1_embeddings.shape)
  dire_pos2_embeddings = hero_embeddings['dire_pos2'].detach().numpy().reshape(-1)
  #print(dire_pos2_embeddings.shape)
  dire_pos3_embeddings = hero_embeddings['dire_pos3'].detach().numpy().reshape(-1)
  #print(dire_pos3_embeddings.shape)
  dire_pos4_embeddings = hero_embeddings['dire_pos4'].detach().numpy().reshape(-1)
  #print(dire_pos4_embeddings.shape)
  dire_pos5_embeddings = hero_embeddings['dire_pos5'].detach().numpy().reshape(-1)
  #print(dire_pos5_embeddings.shape)
  # ... get embedding vectors for other positions

  # Extract the additional feature columns from your dataframe
  rad_pos1_last_gold = df['rad_pos1_last_gold'].values
  rad_pos2_last_gold = df['rad_pos2_last_gold'].values
  rad_pos3_last_gold = df['rad_pos3_last_gold'].values
  rad_pos4_last_gold = df['rad_pos4_last_gold'].values
  rad_pos5_last_gold = df['rad_pos5_last_gold'].values

  dire_pos1_last_gold = df['dire_pos1_last_gold'].values
  dire_pos2_last_gold = df['dire_pos2_last_gold'].values
  dire_pos3_last_gold = df['dire_pos3_last_gold'].values
  dire_pos4_last_gold = df['dire_pos4_last_gold'].values
  dire_pos5_last_gold = df['dire_pos5_last_gold'].values

  pos1_diff = df['pos1_diff'].values
  pos2_diff = df['pos2_diff'].values
  pos3_diff = df['pos3_diff'].values
  pos4_diff = df['pos4_diff'].values
  pos5_diff = df['pos5_diff'].values
  team_diff = df['team_diff'].values

  winner = df['winner'].values

  # combined_features = np.concatenate((rad_pos1_embeddings, rad_pos2_embeddings, rad_pos3_embeddings, rad_pos4_embeddings, rad_pos5_embeddings,
  #                                     dire_pos1_embeddings, dire_pos2_embeddings, dire_pos3_embeddings, dire_pos4_embeddings, dire_pos5_embeddings,
  #                                     rad_pos1_last_gold.reshape(-1, 1), rad_pos2_last_gold.reshape(-1, 1), rad_pos3_last_gold.reshape(-1, 1), rad_pos4_last_gold.reshape(-1, 1), rad_pos5_last_gold.reshape(-1, 1),
  #                                     dire_pos1_last_gold.reshape(-1, 1), dire_pos2_last_gold.reshape(-1, 1), dire_pos3_last_gold.reshape(-1, 1), dire_pos4_last_gold.reshape(-1, 1), dire_pos5_last_gold.reshape(-1, 1),
  #                                     pos1_diff.reshape(-1, 1), pos2_diff.reshape(-1, 1), pos3_diff.reshape(-1, 1), pos4_diff.reshape(-1, 1), pos5_diff.reshape(-1, 1),
  #                                     team_diff.reshape(-1, 1), winner.reshape(-1,1)), axis=1)
  combined_features = np.concatenate((rad_pos1_embeddings.reshape(-1, 1), rad_pos2_embeddings.reshape(-1, 1), rad_pos3_embeddings.reshape(-1, 1), rad_pos4_embeddings.reshape(-1, 1), rad_pos5_embeddings.reshape(-1, 1),
                                     dire_pos1_embeddings.reshape(-1, 1), dire_pos2_embeddings.reshape(-1, 1), dire_pos3_embeddings.reshape(-1, 1), dire_pos4_embeddings.reshape(-1, 1), dire_pos5_embeddings.reshape(-1, 1),
                                     rad_pos1_last_gold.reshape(-1, 1), rad_pos2_last_gold.reshape(-1, 1), rad_pos3_last_gold.reshape(-1, 1), rad_pos4_last_gold.reshape(-1, 1), rad_pos5_last_gold.reshape(-1, 1),
                                     dire_pos1_last_gold.reshape(-1, 1), dire_pos2_last_gold.reshape(-1, 1), dire_pos3_last_gold.reshape(-1, 1), dire_pos4_last_gold.reshape(-1, 1), dire_pos5_last_gold.reshape(-1, 1),
                                     pos1_diff.reshape(-1, 1), pos2_diff.reshape(-1, 1), pos3_diff.reshape(-1, 1), pos4_diff.reshape(-1, 1), pos5_diff.reshape(-1, 1),
                                     team_diff.reshape(-1, 1), winner.reshape(-1, 1)), axis=1)


    # Combine the column names
  column_names = ['rad_pos1','rad_pos2','rad_pos3','rad_pos4','rad_pos5',
                  'dire_pos1','dire_pos2','dire_pos3','dire_pos4','dire_pos5',
                  'rad_pos1_last_gold','rad_pos2_last_gold', 'rad_pos3_last_gold','rad_pos4_last_gold','rad_pos5_last_gold',
                  'dire_pos1_last_gold', 'dire_pos2_last_gold', 'dire_pos3_last_gold', 'dire_pos4_last_gold', 'dire_pos5_last_gold',
                  'pos1_diff','pos2_diff','pos3_diff','pos4_diff','pos5_diff','team_diff', 'winner']

    # Convert the combined_features array to a DataFrame with the specified column names
  combined_features_df = pd.DataFrame(combined_features, columns=column_names)


  return combined_features

df_rf_15_embed = get_hero_embedding_and_concat_features(df_rf_15)


In [ ]:
def get_hero_embedding_and_concat_features(df):
    # Extract the hero_ids for each position from your dataframe
    hero_columns = ['rad_pos1', 'rad_pos2', 'rad_pos3', 'rad_pos4', 'rad_pos5',
                    'dire_pos1', 'dire_pos2', 'dire_pos3', 'dire_pos4', 'dire_pos5']

    hero_embeddings = {col: iterate_ids(df[col].values) for col in hero_columns}

    # Convert the embedding tensors to lists or numpy arrays
    for col in hero_columns:
        hero_embeddings[col] = hero_embeddings[col].detach().numpy().tolist()

    # Create a new DataFrame with the embeddings and additional features
    combined_features_df = pd.DataFrame({
        'rad_pos1': hero_embeddings['rad_pos1'],
        'rad_pos2': hero_embeddings['rad_pos2'],
        'rad_pos3': hero_embeddings['rad_pos3'],
        'rad_pos4': hero_embeddings['rad_pos4'],
        'rad_pos5': hero_embeddings['rad_pos5'],
        'dire_pos1': hero_embeddings['dire_pos1'],
        'dire_pos2': hero_embeddings['dire_pos2'],
        'dire_pos3': hero_embeddings['dire_pos3'],
        'dire_pos4': hero_embeddings['dire_pos4'],
        'dire_pos5': hero_embeddings['dire_pos5'],
        'rad_pos1_last_gold': df['rad_pos1_last_gold'],
        'rad_pos2_last_gold': df['rad_pos2_last_gold'],
        'rad_pos3_last_gold': df['rad_pos3_last_gold'],
        'rad_pos4_last_gold': df['rad_pos4_last_gold'],
        'rad_pos5_last_gold': df['rad_pos5_last_gold'],
        'dire_pos1_last_gold': df['dire_pos1_last_gold'],
        'dire_pos2_last_gold': df['dire_pos2_last_gold'],
        'dire_pos3_last_gold': df['dire_pos3_last_gold'],
        'dire_pos4_last_gold': df['dire_pos4_last_gold'],
        'dire_pos5_last_gold': df['dire_pos5_last_gold'],
        'pos1_diff': df['pos1_diff'],
        'pos2_diff': df['pos2_diff'],
        'pos3_diff': df['pos3_diff'],
        'pos4_diff': df['pos4_diff'],
        'pos5_diff': df['pos5_diff'],
        'team_diff': df['team_diff'],
        'winner': df['winner']
    })

    return combined_features_df

df_rf_15_embed = get_hero_embedding_and_concat_features(df_rf_15)

In [ ]:
df_rf_15_embed.shape



(995, 27)

Split RF Structures for Model Training

In [ ]:
# from sklearn.datasets import load_iris
def split_data(df, test_size):
  X = df.drop('winner', axis=1)
  y = df['winner']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

  return X_train, X_test, y_train, y_test



In [ ]:
for i in range(15, 50, 5): # range does not include the last value i.e. 50 wont be included
    df_name = f'df_rf_{i}'
    X_train_name = f'rf_{i}_X_train'
    X_test_name = f'rf_{i}_X_test'
    y_train_name = f'rf_{i}_y_train'
    y_test_name = f'rf_{i}_y_test'
    test_size = .30

    exec(f"{X_train_name}, {X_test_name}, {y_train_name}, {y_test_name} = split_data({df_name},{test_size})")

In [ ]:
rf_15_X_train, rf_15_X_test, rf_15_y_train, rf_15_y_test = split_data(df_rf_15_embed, test_size = .30)

In [ ]:
rf_15_y_train

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et_classifier = ExtraTreesClassifier(n_estimators=100, random_state=42)
et_classifier.fit(rf_15_X_train, rf_15_y_train)

ValueError: setting an array element with a sequence.

In [ ]:
# Create an instance of the random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the random forest classifier with the combined features and labels
rf_classifier.fit(rf_15_X_train, rf_15_y_train)

Unedited Vanilla Code from Local Drive

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Prepare your data
data = pd.DataFrame({
    'hero_id': [...],  # List of hero IDs
    'team': [...],  # List of team identifiers (e.g., 'Radiant' or 'Dire')
    'team_composition': [...],  # List of team compositions (e.g., ['hero1', 'hero2', 'hero3', 'hero4', 'hero5'])
    'forecasted_gold': [...],  # List of forecasted gold values from the LSTM
    'winner': [...]  # List of match winners (e.g., 'Radiant' or 'Dire')
})

# If we predicted win likelhood at every minute from minute 11 to 45, we would need to create a dataset to train on that cuts the time-series off at each minute
# Let's start off by creating datasets that chop time-series and keep the win at minutes  15, 20, 25, 30, 35

# So if we wanted to predict a winner at minute 11, we would forecast out 15-11, 4 steps, and use those forecasted gold values

#rad_pos1 - rad_pos2 - rad_pos3 - rad_pos4 - rad_pos5 - rad_fg1 - rad_fg2 - rad_fg3 - rad_fg4 - rad_fg5 - <insert dire values>
# include differentials between positions?

#1. Check df2 and see if positions are available and if not, re-pull the data to include it





# Split the data into features (X) and target (y)
X = data[['hero_id', 'team', 'team_composition', 'forecasted_gold']]
y = data['winner']

# Encode categorical variables (team and team_composition) using one-hot encoding
X = pd.get_dummies(X, columns=['team', 'team_composition'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a RandomForest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")